In [1]:
from piano_reduction import tools as pr, features as ft, compute_features as cf
from piano_reduction.score_data import ScoreData
import imp
imp.reload(pr)
imp.reload(ft)
imp.reload(cf)

<module 'piano_reduction.compute_features' from '/home/szeto/Documents/fyp/piano-reduction-main/piano_reduction/compute_features.py'>

In [24]:
data = ScoreData.load('score_data/cosi_2.pkl')
testing_data, training_data = data.split()

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

x_train, y_train = training_data.generate_data(length=-1)
x_test, y_test = testing_data.generate_data(length=-1)
lr = LogisticRegression().fit(x_train, y_train)
nb = MultinomialNB().fit(x_train, y_train)
#nn.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)

#x_train, y_train = training_data.generate_data(length=10)
#x_test, y_test = testing_data.generate_data(length=10)
#lstm.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)
class KeepAll():
    @classmethod
    def predict(cls, x):
        return [1 for _ in range(x.shape[0])]
class KeepSome():
    @classmethod
    def predict(cls, x):
        from random import randint
        return [randint(0, 1) for _ in range(x.shape[0])]

In [25]:
data = ScoreData.load('score_data/2.pkl')
validation_data, _ = data.split(0.5)
from sklearn import metrics
print('%30s %11s %11s %11s %11s' % ('', 'Jaccard Similarity', 'Ignore Octave', 'Accuracy' , 'F1 Score'))
models = [('Keep All', KeepAll, -1),
          ('Keep Some', KeepSome, -1),
          ('Logistic Regression', lr, -1), 
          ('Naive Bayes', nb, -1)]
for name, model, length in models:
    validation_data.get_y_pred(model, length, threshold=0.5)
    tmptrain = validation_data.to_binary('y_train')
    tmppred = validation_data.to_binary('y_pred')
    s1 = set()
    t1 = set()
    s2 = set()
    t2 = set()
    for i in range(0,len(tmptrain)):
        for j in range(0,128):
            if tmptrain[i][j]==1:
                t1.add((i,j))
                t2.add((i,j%12))
    for i in range(0,len(tmppred)):
        for j in range(0,128):
            if tmppred[i][j]==1:
                s1.add((i,j))
                s2.add((i,j%12))
    print('%30s %.9f %.9f    %.9f %.9f' % (name , len(s1&t1)*1.0/(len(s1) + len(t1)- len(s1&t1)) , len(s2&t2)*1.0/(len(s2) + len(t2)- len(s2&t2)),
                              metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                              metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred'])))

                               Jaccard Similarity Ignore Octave    Accuracy    F1 Score
                      Keep All 0.501288660 0.729216152    0.501288660 0.667811159
                     Keep Some 0.296357616 0.510256410    0.452319588 0.457215837
           Logistic Regression 0.484507042 0.712895377    0.528350515 0.652751423
                   Naive Bayes 0.494722955 0.723809524    0.506443299 0.661959400


In [26]:
new_data = data.copy()
new_data.get_y_pred(lr, length=-1)
new_data.df['y_train'] = new_data.df['y_pred']
new_data.df = new_data.df.query("(y_train == 1) or (x_train == 1)")
del new_data.df['y_pred']
testing_data, training_data = new_data.split()
x_train, y_train = training_data.generate_data(length=-1)
x_test, y_test = testing_data.generate_data(length=-1)
lr = LogisticRegression().fit(x_train, y_train)
nb = MultinomialNB().fit(x_train, y_train)
nn.fit(x_train, y_train, epochs=1500, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop], verbose=0)
validation_data, _ = new_data.split(0.5)
validation_data = new_data
validation_data.get_y_pred(lr, length=-1,threshold = 0.5)
tmptrain = validation_data.to_binary('y_train')
tmppred = validation_data.to_binary('y_pred')

print('%30s %11s %11s %11s %11s' % ('', 'Jaccard Similarity', 'Ignore Octave', 'Accuracy' , 'F1 Score'))
models = [('Keep All', KeepAll, -1),
          ('Keep Some', KeepSome, -1),
          ('Logistic Regression', lr, -1), 
          ('Naive Bayes', nb, -1),
          ('NN', nn, -1)
         ]
for name, model, length in models:
    validation_data.get_y_pred(model, length, threshold=0.5)
    tmptrain = validation_data.to_binary('y_train')
    tmppred = validation_data.to_binary('y_pred')
    s1 = set()
    t1 = set()
    s2 = set()
    t2 = set()
    for i in range(0,len(tmptrain)):
        for j in range(0,128):
            if tmptrain[i][j]==1:
                t1.add((i,j))
                t2.add((i,j%12))
    for i in range(0,len(tmppred)):
        for j in range(0,128):
            if tmppred[i][j]==1:
                s1.add((i,j))
                s2.add((i,j%12))
    print('%30s %.9f %.9f    %.9f %.9f' % (name , len(s1&t1)*1.0/(len(s1) + len(t1)- len(s1&t1)) , len(s2&t2)*1.0/(len(s2) + len(t2)- len(s2&t2)),
                              metrics.accuracy_score(validation_data.df['y_train'], validation_data.df['y_pred']), 
                              metrics.f1_score(validation_data.df['y_train'], validation_data.df['y_pred'])))


/home/szeto/Documents/fyp/piano-reduction-main/piano_reduction/score_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.df['y_pred'] = tmp


                               Jaccard Similarity Ignore Octave    Accuracy    F1 Score
                      Keep All 0.811603651 0.929133858    0.811603651 0.896005757
                     Keep Some 0.453247680 0.669365722    0.500651890 0.623772102
           Logistic Regression 0.971653543 0.983240223    0.976531943 0.985623003
                   Naive Bayes 0.800738007 0.904183536    0.823989570 0.889344262
                            NN 0.946750196 0.986033520    0.955671447 0.972646822


In [8]:
validation_data.save('intermediate/2.pkl')

In [27]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras import backend as K
from keras.callbacks import BaseLogger
from keras.callbacks import EarlyStopping
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

early_stop = EarlyStopping(monitor='val_acc', mode='max', min_delta=0, patience=10, verbose=0, restore_best_weights=True)

In [28]:
nn = Sequential()
nn.add(Dense(200, input_shape=[cf.feature_count()]))
nn.add(Activation('sigmoid'))
nn.add(Dense(1))
nn.add(Activation('sigmoid'))
nn.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', f1])
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 200)               3200      
_________________________________________________________________
activation_3 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [9]:
data = ScoreData.load('intermediate/2.pkl')

In [29]:
data_pred = data.to_binary('y_pred')
tmp = data_pred

In [17]:
new_data.show_score(col=['x_train', 'y_train', 'y_pred', 'post'])

In [30]:
for i in range(0,len(data_pred)):
    cnt = 0
    highest = -1
    lowest = 129
    for j in range(0,128):
        if data_pred[i][j] == 1:
            highest = max(highest,j)
            lowest = min(lowest,j)
    for j in range(60,128):
        if data_pred[i][j] ==1 and j < highest-12:
            data_pred[i][j] = 0
            data_pred[i][j+12] = 1
    for j in reversed(range(0, 60)):
        if data_pred[i][j] == 1 and j > lowest + 12:
            data_pred[i][j] = 0
            data_pred[i][j - 12] = 1

In [6]:
data_pred

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [114]:
from music21 import environment
environment.set('musicxmlPath', '/usr/bin/musescore')

In [15]:
new_data = data.merge_binary(data_pred, name='post', skip_features=True)